In [1]:
#pragma cling add_include_path("./thirdparty/kdb/include")
#pragma cling add_include_path("./thirdparty/kdb/include/internal")
#pragma cling load("./thirdparty/kdb/build/libkdb.so")
#pragma cling add_include_path("./thirdparty/libtorch/include")
#pragma cling add_include_path("./thirdparty/libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("./thirdparty/libtorch/lib")
#pragma cling load("libtorch")

#pragma cling add_include_path("./thirdparty/nest/include")

#include <torch/torch.h>
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

#include "xtensor/xbuilder.hpp"
#include "xtensor/xarray.hpp"
#include "xtensor/xtensor.hpp"
#include "nest.h"
#include <vector>
#include <array>

#include </home/thorad/Core/Projects/Qenv/notebooks/thirdparty/kdb/include/kdb_cpp.h>
#include <string>
#include <iostream>
#include <typeinfo>

kdb::Connector kcon;
#define HOST_ADDR "127.0.0.1"
#define HOST_PORT 5050

if (!kcon.connect(HOST_ADDR, HOST_PORT))
        return -1;

kdb::Result rs0 = kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));0]");
kdb::Result rs1 = kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));1]");

K rks0 = rs0.get_res();
K rks1 = rs1.get_res();

// std::cout << rs1 << std::endl;
typedef nest::Nest<torch::Tensor> TensorNest;

In [ ]:
std::vector<kdb::Result> responses = {};

In [ ]:
%timeit -n 1000 responses.push_back(kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5));0]"));

In [2]:
K w = kK(kK(rks1)[0])[1];
F* x = kF(w);
std::vector<double> y = {};
for (int i=0;i<256;i++){
    std::cout << x[i] << std::endl;
    y.push_back(double(x[i]));
}

219.702
126.037
81.4733
114.779
16.8625
129.7
65.0318
32.2658
173.166
222.992
131.195
83.1956
91.49
234.199
130.605
154.92
115.472
162.892
67.5007
214.389
12.9624
127.145
160.975
191.822
7.42523
186.482
45.1753
162.884
250.153
124.698
85.6072
22.3788
86.9495
121.548
157.086
162.614
77.903
63.5826
153.47
116.931
44.3456
195.593
98.4595
120.886
189.75
69.8638
222.939
242.926
132.608
35.5573
15.0482
117.752
227.777
60.58
2.70725
47.1829
78.7791
233.828
84.2124
208.205
186.012
93.0623
11.0049
124.138
35.7167
42.958
195.586
64.9862
128.281
118.37
213.173
41.6905
66.8762
80.6493
74.8896
226.652
29.8936
238.242
121.907
216.266
161.298
196.324
33.6456
57.578
24.0449
182.25
120.042
132.42
201.631
128.372
77.7556
188.117
65.1919
97.9065
21.1895
191.997
137.796
137.465
231.131
137
251.1
129.687
20.6267
151.092
177.763
87.2117
156.885
202.096
63.7003
92.7538
53.1719
168.654
143.363
39.155
79.8686
216.195
209.51
253.889
33.2118
44.2919
63.3695
16.8233
192.242
17.6348
238.762
121.518
111.952
157.219

In [4]:
%timeit -n 1000 -r 10 std::vector({x}).data();

121 ns +- 17.1 ns per loop (mean +- std. dev. of 10 runs 1000 loops each)


In [ ]:
torch::CPU(torch::kFloat32)

In [2]:
double array[] = { 1, 2, 3, 4, 5};
auto options = torch::TensorOptions().dtype(torch::kFloat64).device(torch::kCUDA, 1);
torch::Tensor tharray = torch::from_blob(array, {5}, options);

IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt11__once_call' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!
IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt15__once_callable' unresolved while linking function '_GLOBAL__sub_I_cling_module_7'!


In [3]:
torch::Tensor tharray = torch::tensor({1, 2, 3, 4, 5}, {torch::kFloat64});

In [ ]:
torch::from_blob(std::vector({0, 1, 1, 1, 0}).data(), {5}, torch::kI32)

In [ ]:
// `agentId`observation`reward`done`episode_step`episode_return
std::vector<TensorNest> derive_multistep_from_result(kdb::Result& result){
    K kres  = result.get_res();
    std::vector<TensorNest> tensors;
    for (int i=0; i< kres->n; i++) {
        K step = kK(kres)[0]; // TODO
        tensors.push_back(TensorNest(std::vector({ // omits agentId
            std::move(TensorNest(torch::tensor(kF(kK(step)[1]), {torch::kFloat64}))), // observation // TODO
            std::move(TensorNest(torch::tensor(kK(step)[2]->f, {torch::kFloat64}))), // reward
            std::move(TensorNest(torch::tensor(kK(step)[3]->g, {torch::kBool}))), // done
            std::move(TensorNest(torch::tensor(kK(step)[4]->f, {torch::kFloat64}))), // episode_step TODO change
            std::move(TensorNest(torch::tensor(kK(step)[5]->f, {torch::kFloat64})))  // episode_return
            })));
    };
    return tensors;
};

In [3]:
%timeit -n 10000 -r 10 std::vector<TensorNest> res = derive_multistep_from_result(rs1);

82.8 us +- 466 ns per loop (mean +- std. dev. of 10 runs 10000 loops each)


In [ ]:
std::cout << res.size() << std::endl;

In [ ]:
torch::tensor({}, {torch::kFloat64})

In [4]:
// `agentId`observation`reward`done`episode_step`episode_return
std::vector<TensorNest> derive_multistep_from_resultx(kdb::Result& result){
    K kres  = result.get_res();
    std::vector<TensorNest> tensors;
    for (int i=0; i< kres->n; i++) {
        K step = kK(kres)[0]; // TODO
        tensors.push_back(TensorNest(std::vector({ // omits agentId
            std::move(TensorNest(torch::tensor(kF(kK(step)[1]), {torch::kFloat64}))), // observation // TODO
            std::move(TensorNest(torch::full({1,1},kK(step)[2]->f,torch::dtype(torch::kFloat64)))), // reward
            std::move(TensorNest(torch::full({1,1},kK(step)[3]->g,torch::dtype(torch::kBool)))), // done
            std::move(TensorNest(torch::full({1,1},kK(step)[4]->f,torch::dtype(torch::kFloat64)))), // episode_step TODO change
            std::move(TensorNest(torch::full({1,1},kK(step)[5]->f,torch::dtype(torch::kFloat64))))  // episode_return
            })));
    };
    return tensors;
};

In [5]:
%timeit -n 10000 -r 10 std::vector<TensorNest> res = derive_multistep_from_resultx(rs1);

91.8 us +- 385 ns per loop (mean +- std. dev. of 10 runs 10000 loops each)


In [ ]:
kdb::Table tbl = res.get_table();

In [ ]:
// Traverse a column
kdb::Vector<kdb::Type::Long> column = tbl.get_column<kdb::Type::Long>(0);
std::cout << "=======================================\nColumn 0: ";
for (auto const &it : column) {
    std::cout << it << ' ';
}
std::cout << '\n';

In [ ]:
std::cout << torch::full({1,1},1) << std::endl;

In [ ]:
typedef nest::Nest<torch::Tensor> TensorNest;

In [ ]:
torch::Tensor x = torch::ones({2, 2}, torch::requires_grad());
torch::Tensor y = torch::randn({2, 2});

In [ ]:
std::vector<size_t> shape = { 3, 2, 4 };
std::vector<size_t> strides = { 8, 4, 1 };

In [ ]:
std::vector<size_t> shape = { 3, 2, 4 };
xt::xarray<double, xt::layout_type::row_major> a(shape);

In [ ]:
std::array<size_t, 3> shape = { 3, 2, 4 };
xt::xtensor<double, 3> b(shape);

In [ ]:
// typedef int  I;          // Already defined within k.h
// typedef char *S;         // Already defined within k.h
// I khpu(S hostname, I portNumber, S usernamePassword)
khpu ("localhost", 12345, "username:password");